# Conjoint Analysis 
Calculates utilities, price units, and perceived values for each product/service from product/services part-worths from a conjoint analysis data file (JSON), outputs Excel tables.

Example analysis estimates customers' perceived value (willingness to pay) for Trailblazers game packages.

John Bonfardeci 2017

In [1]:
import pandas as pd
from IPython.display import display
import numpy as np

save_as = 'trailblazers'
file_path = 'trailblazers.csv'
df = pd.read_csv(file_path)

display(df.head())

,Attribute,Level,Partworth,Cost,Price
0,Number of Games,3-Game,0.03257,NaN,NaN
1,Number of Games,6-Game,0.24383,NaN,NaN
2,Number of Games,10-Game,-0.27640,NaN,NaN
3,Ticket Price,$15/seat/game,0.65646,10.0,15.0
4,Ticket Price,$25/seat/game,0.22011,12.0,25.0


## Functions

In [2]:
def get_diff(a):
    return max(a) - min(a)


def get_attr_importance(pw_diff, total_pw):
    return pw_diff / total_pw * 100


def get_attr_util(df):
    df = df.copy()
    price_unit = None
    
    # calc the part-worth difference from all levels, max - min
    partworths = [r for r in df['Partworth']]
    _min = min(partworths)
    prices = [r for r in df['Price']]
    pw_diff = get_diff( partworths )
    
    # calc price utility of each level
    util = lambda pw: (pw - _min) if (pw != _min) else pw
    utilities = [util(pw) for pw in partworths]
    df['Utility'] = utilities
    
    if not np.isnan(prices[0]):
        price_diff = get_diff(prices)
        price_unit = price_diff / pw_diff
        
    df['PriceUnit'] = [price_unit for r in prices]
    
    return [df, pw_diff]


def get_attr_values(df, price_unit):
    """
    Return dataframe with calculated perceived values 
    from part-worths and utilities. 
    """
    perc_values = []   
    get_perc_val = lambda u: u * price_unit
    perc_values = [get_perc_val(u) for u in df['Utility']]
    return perc_values

def get_wtp(df):
    price_unit = None
    pw_total = 0.00
    attrs_list = []
    imp_data = []
    
    attrs = df.groupby(['Attribute'])
    attr_names = [a[0] for a in attrs]
    
    for attr in attrs:
        attr_df, pw_diff = get_attr_util(attr[1])
        pw_total += pw_diff
        attrs_list.append(attr_df)
        #display(attr_df.head())
        
        if 'PriceUnit' in attr_df.columns:
            price_unit = attr_df['PriceUnit'].as_matrix()[0]
                 
    for attr, name in zip(attrs_list, attr_names):
        attr['WTP'] = get_attr_values(attr, price_unit)
        u_diff = get_diff( [u for u in attr['Utility'].as_matrix()] )
        imp_data.append( (name, u_diff, get_attr_importance(u_diff, pw_total) ) )
    
    df_imp = pd.DataFrame(data=imp_data, columns=['Attribute', 'Utility', 'Importance'])\
                      .sort_values(by='Importance', ascending=False)
        
    return [attrs_list, df_imp]

## Import data and calculate WTP.

In [3]:
attrs, imp = get_wtp(df)

imp.to_csv(save_as+'_importance.csv')
display(imp)

merged = pd.concat(attrs).sort_index()
merged.to_csv(save_as+'_values.csv')
display(merged)

pivot = merged.T
pivot.to_csv(save_as+'_pivot.csv')
display(pivot)

,Attribute,Utility,Importance
3,Ticket Price,2.66160,60.291263
2,Ticket Location,2.47486,56.061179
1,Promo Item,0.81000,18.348333
0,Number of Games,0.79663,18.045472


,Attribute,Level,Partworth,Cost,Price,Utility,PriceUnit,WTP
0,Number of Games,3-Game,0.03257,NaN,NaN,0.30897,None,8.380590
1,Number of Games,6-Game,0.24383,NaN,NaN,0.52023,None,14.110866
2,Number of Games,10-Game,-0.27640,NaN,NaN,-0.27640,None,-7.497152
3,Ticket Price,$15/seat/game,0.65646,10.00,15.0,1.65903,27.1243,45.000000
4,Ticket Price,$25/seat/game,0.22011,12.00,25.0,1.22268,27.1243,33.164319
5,Ticket Price,$35/seat/game,0.12600,18.00,35.0,1.12857,27.1243,30.611653
6,Ticket Price,$60/seat/game,-1.00257,40.00,60.0,-1.00257,27.1243,-27.193993
7,Ticket Location,300BB,-0.73169,10.00,NaN,-0.73169,None,-19.846567
8,Ticket Location,300C,0.43716,12.00,NaN,1.16885,None,31.704219
9,Ticket Location,300M,0.15736,18.00,NaN,0.88905,None,24.114844


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Attribute,Number of Games,Number of Games,Number of Games,Ticket Price,Ticket Price,Ticket Price,Ticket Price,Ticket Location,Ticket Location,Ticket Location,Ticket Location,Promo Item,Promo Item,Promo Item,Promo Item,Promo Item
Level,3-Game,6-Game,10-Game,$15/seat/game,$25/seat/game,$35/seat/game,$60/seat/game,300BB,300C,300M,200M,Priority Playoff Tickets,Dog n Pop,Apparel,$20 Certificate,No Promo
Partworth,0.03257,0.24383,-0.2764,0.65646,0.22011,0.126,-1.00257,-0.73169,0.43716,0.15736,1.01148,0.12511,0.17428,0.00158,0.01689,-0.31786
Cost,NaN,NaN,NaN,10,12,18,40,10,12,18,40,0,3.25,12,10,0
Price,NaN,NaN,NaN,15,25,35,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Utility,0.30897,0.52023,-0.2764,1.65903,1.22268,1.12857,-1.00257,-0.73169,1.16885,0.88905,1.74317,0.44297,0.49214,0.31944,0.33475,-0.31786
PriceUnit,None,None,None,27.1243,27.1243,27.1243,27.1243,None,None,None,None,None,None,None,None,None
WTP,8.38059,14.1109,-7.49715,45,33.1643,30.6117,-27.194,-19.8466,31.7042,24.1148,47.2822,12.0152,13.3489,8.66458,9.07985,-8.62172
